# Readability Assessment through Learning Techniques

* Readability assessment is a well known problem in natural language processing field. 
* Giving someone the suitable text for his level of comprehension (not so easy and not so hard) could maximize his understanding and enjoyment. 

In this notebook we are trying to assess the readability of a given text regardless of the text topic.

## Corpus
> OneStopEnglish corpus: A new corpus for automatic readability assessment and text simplification  
> Sowmya Vajjala and Ivana Lučić  
> 2018  
> Proceedings of the Thirteenth Workshop on Innovative Use of NLP for Building Educational Applications, pages 297–304. Association for Computational Linguistics.  
> [url](http://aclweb.org/anthology/W18-0535). [bib file](https://aclanthology.coli.uni-saarland.de/papers/W18-0535/w18-0535.bib)

Please cite the above paper if you use this corpus in your research.

[![DOI](https://zenodo.org/badge/128919409.svg)](https://zenodo.org/badge/latestdoi/128919409)

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

Now let's dive into our corpus.

In [2]:
import sys
sys.path.append("/home/ms10596/PycharmProjects/match")
from ipywidgets import interact
from tabulate import tabulate
from IPython.display import display, HTML

from utils.loading import load_glove_embeddings, load_old_corpus
from utils.one_stop_english import load_corpus, corpus_to_words, corpus_to_pos, detokenize

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional,Conv1D,MaxPooling1D,GlobalMaxPooling1D, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np

corpus = load_corpus()
articles_words, tags = corpus_to_words(corpus)
articles_pos, tags = corpus_to_pos(corpus)
old_articles_pos, old_articles_tags = load_old_corpus()

Reading level|Avg. Num. Words|Std. Dev|Number of Articles
---|---|---|---
Elementary|533.17|103.79|189
Intermediate|676.59|117.15|189
Advanced|820.49|162.52|189



In [3]:
@interact
def show_articles(i=(0,188,1), words=(0,1000,1)):
    data = [
        ["Advanced",detokenize(articles_words[i][:words])], 
        ["Intermediate",detokenize(articles_words[i+2][:words])], 
        ["Elementary",detokenize(articles_words[i+1][:words])]
    ]
    headers = ['Reading Level', 'Example']
    display(HTML(tabulate(data,tablefmt='html', headers=headers)+"<style>th,td {font-size: 20px}</style>"))

interactive(children=(IntSlider(value=94, description='i', max=188), IntSlider(value=500, description='words',…

In [4]:
print(articles_words[0][:10])
print(articles_pos[0][:10])

['When', 'you', 'see', 'the', 'word', 'Amazon', ',', 'whats', 'the', 'first']
['WRB', 'PRP', 'VB', 'DT', 'NN', 'NN', ',', 'VBZ', 'DT', 'JJ']


In [5]:
import json
maxlen = 1000 # Cuts off reviews after 1000 words
max_words = 45
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(articles_pos)

with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer.to_json(), ensure_ascii=False))
print(tokenizer.word_index)

{'nn': 1, 'in': 2, 'dt': 3, 'jj': 4, 'nns': 5, 'nnp': 6, ',': 7, '.': 8, 'rb': 9, 'prp': 10, 'vb': 11, 'vbd': 12, 'cc': 13, 'vbz': 14, 'to': 15, 'vbp': 16, 'cd': 17, 'vbn': 18, 'vbg': 19, 'prp$': 20, 'md': 21, 'wdt': 22, 'wrb': 23, 'wp': 24, 'jjr': 25, 'rp': 26, ':': 27, 'jjs': 28, 'ex': 29, 'rbr': 30, 'nnps': 31, '-rrb-': 32, '-lrb-': 33, 'rbs': 34, 'pdt': 35, '$': 36, 'fw': 37, 'uh': 38, 'wp$': 39, 'sym': 40, "''": 41, 'ls': 42, 'pos': 43, '``': 44}


In [7]:
sequences = tokenizer.texts_to_sequences(articles_pos)
old_sequences = tokenizer.texts_to_sequences(old_articles_pos)
print(np.shape(sequences))
print(np.shape(old_sequences))
# print(sequences[0])

(567,)
(183,)


In [8]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
old_data = pad_sequences(old_sequences, maxlen=maxlen, padding='post', truncating='post')
# print(data[0])
print(np.shape(data))
print(np.shape(old_data))

(567, 1000)
(183, 1000)


In [11]:
print(tags[0])
print(old_articles_tags[0])
tags = np.array(tags)
old_tags = np.array(old_articles_tags)
print(tags.shape)
print(old_tags.shape)

2
0
(567,)
(183,)


In [12]:
data = to_categorical(data)
old_data = to_categorical(old_data)
print(data.shape)
print(old_data.shape)

(567, 1000, 45)
(183, 1000, 45)


In [16]:
np.random.seed(0)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
tags = tags[indices]

old_indices = np.arange(old_data.shape[0])
np.random.shuffle(indices)

old_data = old_data[old_indices]
old_tags = old_tags[old_indices]
print(data.shape)
print(tags.shape)

print(old_data.shape)
print(old_tags.shape)

(567, 1000, 45)
(567,)
(183, 1000, 45)
(183,)


In [20]:
model = Sequential()
model.add(Conv1D(128, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.01, l2=0.01), input_shape=(1000,45)))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
# model.add(MaxPooling1D(5))
# model.add(Conv1D(32, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.0003, l2=0.0003)))
model.add(GlobalMaxPooling1D())
model.add(Dense(1))
# model.summary()
def soft_acc(y_true, y_pred):
    from tensorflow.python.keras import backend as K
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [29]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvs_scores = []
for train, test in kfold.split(data, tags):
    model = Sequential()
    model.add(Conv1D(128, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.01, l2=0.01), input_shape=(1000,45)))
    model.add(MaxPooling1D(5))
    model.add(Conv1D(64, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
    model.add(MaxPooling1D(5))
    model.add(Conv1D(32, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.0003, l2=0.0003)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1))
    # model.summary()
    def soft_acc(y_true, y_pred):
        from tensorflow.python.keras import backend as K
        return K.mean(K.equal(K.round(y_true), K.round(y_pred)))
    model.compile(optimizer=RMSprop(),loss='mse',metrics=[soft_acc])
    model.fit(data[train], tags[train], epochs=2000, verbose=0)
    scores = model.evaluate(data[test], tags[test], verbose=0)
    print(scores[1]*100)
    cvs_scores.append(scores[1]*100)
print("Accuracy:", np.mean(cvs_scores),"std:", np.std(cvs_scores))

61.40350887649938
66.66666792150129
49.122808272378485
45.614034617156314
64.91228080632393
54.385964807711154
57.89473715581393
73.68421178115042
64.91228101546305
55.555556217829384
Accuracy: 59.415205147182725 std: 8.120868247309904


In [ ]:
es = EarlyStopping(monitor='val_soft_acc',patience=100, mode='max')
cp = ModelCheckpoint('model-{epoch:03d}-{soft_acc:03f}-{val_soft_acc:03f}.h5', verbose=1, monitor='val_soft_acc',save_best_only=True, mode='max')  
model.compile(optimizer=RMSprop(),loss='mse',metrics=[soft_acc])
history = model.fit(data, tags,epochs=1000,  batch_size=50,validation_data=(old_data, old_tags),callbacks=[])
# model.save_weights('pre_trained_glove_model.h5')
model.summary()

Train on 567 samples, validate on 183 samples
Epoch 1/1000
567/567 [==============================] - 1s 1ms/step - loss: 12.8269 - soft_acc: 0.3122 - val_loss: 10.4141 - val_soft_acc: 0.4098
Epoch 2/1000
567/567 [==============================] - 0s 644us/step - loss: 9.0257 - soft_acc: 0.3404 - val_loss: 7.4448 - val_soft_acc: 0.3880
Epoch 3/1000
567/567 [==============================] - 0s 653us/step - loss: 6.5777 - soft_acc: 0.3439 - val_loss: 5.3564 - val_soft_acc: 0.3989
Epoch 4/1000
567/567 [==============================] - 0s 627us/step - loss: 4.6956 - soft_acc: 0.3721 - val_loss: 3.7391 - val_soft_acc: 0.3880
Epoch 5/1000
567/567 [==============================] - 0s 637us/step - loss: 3.2329 - soft_acc: 0.3739 - val_loss: 2.7400 - val_soft_acc: 0.3333
Epoch 6/1000
567/567 [==============================] - 0s 642us/step - loss: 2.2974 - soft_acc: 0.4092 - val_loss: 1.8975 - val_soft_acc: 0.4372
Epoch 7/1000
567/567 [==============================] - 0s 635us/step - loss: 

567/567 [==============================] - 0s 625us/step - loss: 0.7653 - soft_acc: 0.4497 - val_loss: 0.8840 - val_soft_acc: 0.3333
Epoch 57/1000
567/567 [==============================] - 0s 652us/step - loss: 0.7756 - soft_acc: 0.4515 - val_loss: 0.8000 - val_soft_acc: 0.3934
Epoch 58/1000
567/567 [==============================] - 0s 646us/step - loss: 0.7694 - soft_acc: 0.4515 - val_loss: 0.8068 - val_soft_acc: 0.4645
Epoch 59/1000
567/567 [==============================] - 0s 666us/step - loss: 0.7615 - soft_acc: 0.4533 - val_loss: 0.8115 - val_soft_acc: 0.3497
Epoch 60/1000
567/567 [==============================] - 0s 671us/step - loss: 0.7605 - soft_acc: 0.4603 - val_loss: 0.8134 - val_soft_acc: 0.3497
Epoch 61/1000
567/567 [==============================] - 0s 678us/step - loss: 0.7707 - soft_acc: 0.4497 - val_loss: 0.7955 - val_soft_acc: 0.4044
Epoch 62/1000
567/567 [==============================] - 0s 664us/step - loss: 0.7538 - soft_acc: 0.4533 - val_loss: 0.8008 - val_so

In [ ]:
import matplotlib.pyplot as plt

soft_acc = history.history['soft_acc']
soft_val_acc = history.history['val_soft_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(soft_acc) + 1)
plt.figure(figsize=(30,5))
plt.plot(epochs, soft_acc, 'bo', label='Soft Training acc')
plt.plot(epochs, soft_val_acc, 'b', label='Soft Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure(figsize=(30,5))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


